## Validating the model's performance on user specified stocks and date range

In [1]:
import neat
import pandas as pd
import numpy as np
import datetime
import brain
import pickle
from stock_class import Stock
from portfolio_class import Portfolio
import random
import pprint
import matplotlib.pyplot as plt

### Load historical datasets

In [2]:
with open("validation.pickle", "rb") as f:
        validation = pickle.load(f)
    
with open("spy.pickle", "rb") as f:
        spy = pickle.load(f)

### Selecting a subset of stocks and a date range

In [3]:
# Set a start date (2022-5-16 is the earliest available in the validation dataframe)
start_date = datetime.date(2022,7,16)
# Set an end date (2022-10-24 is the latest available in the validation dataframe)
end_date = datetime.date(2022,10,15)
# Set how many stocks to consider for buying/selling every day
num_selected = 40

# Set starting cash
cash = 1000

# Random sample num_selected amount from the available stocks in the dataset
stocks_in_the_dataset = validation['ticker'].unique()
subset_of_stocks = random.choices(stocks_in_the_dataset, k=num_selected)
pp = pprint.PrettyPrinter(width=90, compact=True)
print(f'Selected stocks that will be considered for trading between {start_date} -- {end_date} are: ')
print(" ")
pp.pprint(subset_of_stocks)

Selected stocks that will be considered for trading between 2022-07-16 -- 2022-10-15 are: 
 
['VRTV', 'PASG', 'MCFT', 'BH', 'CVAC', 'UHAL', 'SOI', 'RPRX', 'BNL', 'PEN', 'UPST', 'WRK',
 'BEAM', 'VRTV', 'GRPN', 'VOXX', 'TERN', 'BBSI', 'AGR', 'TRNO', 'RUN', 'PDI', 'SPNT',
 'NDLS', 'TBNK', 'CDLX', 'PUBM', 'RNG', 'DSP', 'ZG', 'GBDC', 'UPST', 'VOR', 'ALGM',
 'SSYS', 'VIR', 'CC', 'ARCC', 'CD', 'AKRO']


### Filter the requested stocks and dates

In [4]:
df_selected = validation[validation['ticker'].isin(subset_of_stocks) & 
                             validation['date_'].isin(
                                 [item.date() for item in list(pd.date_range(start=start_date, end=end_date))]
                             )]
df_selected.head(5)

,close,volume,ticker,sector,sma_9,sma_14,sma_20,sma_50,sma_100,sma_200,...,Communication Services,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology,Utilities
2022-07-18,121.050003,322900.0,PEN,Healthcare,126.363331,126.628570,126.471999,134.123201,169.04910,210.948000,...,0,0,0,0,0,1,0,0,0,0
2022-07-18,15.150000,756700.0,PUBM,Technology,15.933333,16.045714,16.655000,18.726600,21.55350,26.019100,...,0,0,0,0,0,0,0,0,1,0
2022-07-18,2.160000,177900.0,PASG,Healthcare,2.333333,2.351429,2.382500,2.173000,2.54005,4.793325,...,0,0,0,0,0,1,0,0,0,0
2022-07-18,11.160000,289200.0,AKRO,Healthcare,11.163333,10.695714,10.789500,9.683000,11.80340,16.474200,...,0,0,0,0,0,1,0,0,0,0
2022-07-18,18.790001,725900.0,SSYS,Technology,17.741111,18.108572,18.359000,18.470200,20.71490,23.594850,...,0,0,0,0,0,0,0,0,1,0


### Define the main loop

In [5]:
def main(best_genome_name, data, cash):
    
    with open(best_genome_name, "rb") as f:
        best_genome = pickle.load(f)
    
    # Load configuration.
    config_file = r'neat_config.txt'

    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # X_cols and portfolio_attributes
    X_cols = ['cat_1', 'cat_2', 'cat_3', 'Basic Materials', 'Communication Services',
              'Consumer Cyclical', 'Consumer Defensive', 'Energy',
              'Financial Services', 'Healthcare', 'Industrials', 'Real Estate',
              'Technology', 'Utilities', 'variance_100', 'variance_global']
    portfolio_attributes = ["proportion_invested", "entropy_stock", "entropy_sector"]

    # preprocessing
    training_data = data.copy()
    brain.shape_config_file(config_file, X_cols, portfolio_attributes)
    X_cols = [col.lower() for col in X_cols]
    
    threshold = 0.05

    # create 1-member population
    p = neat.Population(config)
    p.population = {1: best_genome}

    # clear and set Stocks
    Stock.clear_stock_list()
    Stock.set_each(training_data)

    # same loop as in brain.run_neat
    for genome_id, genome in p.population.items():
        genome.fitness = 0
        genome_portfolio = Portfolio(genome=genome, cash=cash)
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for current_date in training_data.index.unique():

            # shuffle
            random.shuffle(Stock.stock_list)
            for stock in Stock.stock_list:
                try:
                    df = stock.data.loc[current_date, X_cols]
                    for item in portfolio_attributes:
                        df[item] = genome_portfolio.__getattribute__(item)
                except KeyError:
                    continue
                output = net.activate(list(df))
                decision, proportion = brain.output_to_decision(output[0], threshold=threshold)
                if decision == 'buy':
                    amount = (proportion * genome_portfolio.cash_current) / stock.data.loc[current_date, "close"]
                    genome_portfolio.buy(stock=stock, amount=amount, as_of=current_date)
                elif decision == 'sell':
                    amount = -1 * (proportion * genome_portfolio.get_stock_amount(stock))
                    genome_portfolio.sell(stock=stock, amount=amount, as_of=current_date)
                elif not decision:
                    pass
                else:
                    raise NotImplementedError
            # End-of-day update
            genome_portfolio.update_balance_eod(as_of=current_date)
    
    print("Succesfully simulated trading.")
    return genome, genome_portfolio

### Calculate S&P 500 return during the selected period

In [6]:
first_close = spy.loc[spy.index.date == df_selected.index[0], "Close"]
last_close = spy.loc[spy.index.date == df_selected.index[-1], "Close"]
spy_return = last_close[0] / first_close[0]
print(f"S&P500 total return between {start_date} & {end_date}: ")
print(round(spy_return,4))

S&P500 total return between 2022-07-16 & 2022-10-15: 
0.9363


### Simulate trading

In [ ]:
# Set the best genome
best_genome_name = "best_genome_123.pickle"

# Run main
genome, portfolio = main(best_genome_name=best_genome_name, data=df_selected, cash=cash)

### Calculate our startegy's return

In [ ]:
print(f"Total return of trading bot between {start_date} & {end_date} only trading selected stocks: ")
print(round((portfolio.total_portfolio_value_hist[-1][1] / cash), 4))

### Visualize the difference between S&P500 and our trading strategy

In [ ]:
# Create two separate variables from list of tulples
dates = []
profits = []
for item in portfolio.total_portfolio_value_hist:
    dates.append(item[0])
    profits.append(item[1])

# Calculate the change of S&P500 standardized to match our starting cash    
spy_selected = spy[spy.index.isin([item.date() for item in list(pd.date_range(start=start_date, end=end_date))])]
spy_selected['Value'] = np.nan
spy_selected['Value'][0] = cash
for i in range(1,len(spy_selected['Value'])):
    spy_selected['Value'][i] = spy_selected['Return'][i] * spy_selected['Value'][i-1]

# Create plot
plt.figure(figsize=(12,4))
plt.title("Change of invested value through time")
plt.plot(dates, profits, label="Portfolio value")
plt.plot(dates, spy_selected["Value"], label="S&P500 value" )
plt.legend()

### Final portfolio

In [ ]:
portfolio.balance.head(portfolio.balance.last_valid_index())